<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
چی بخریم؟
</font>
</h1>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
به یک پروژه‌ی متفاوت و کاربردی خوش آمدید! 🤩 در این مسئله گریزی به <i>سیستم‌های پیشنهاددهنده (Recommender Systems)</i> خواهیم زد و با استفاده از یک معماری یادگیری عمیق خلاقانه، مدلی پیاده خواهیم کرد که بتواند علاقه‌ی یک کاربر به یک کالای مشخص را تخمین بزند. با استفاده از این مدل خواهیم توانست لیستی از کالاها را به کاربر پیشنهاد دهیم که تاکنون آن‌ها را خریداری نکرده اما طبق الگوهایی که مدل بین کاربران و کالاها پیدا کرده به‌احتمال زیاد از آن‌ها خوشش خواهد آمد و به آن‌ها امتیاز بالایی خواهد داد. مجموعه‌داده‌ی پروژه نیز شامل داده‌های واقعی مربوط به کالاهای الکترونیکی موجود در <i>فروشگاه آنلاین آمازون (Amazon)</i> است.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="orange"><b>نکته:</b></font>
<font face="vazir" size=3>
اگرچه در این پروژه با داده‌های فروشگاه آنلاین سروکار داریم اما این مدل پیشنهاددهنده را می‌توان در دامنه‌های گوناگونی به کار گرفت. به‌عنوان مثال می‌توان سیستمی مشابه جهت پیشنهاد فیلم یا قطعه‌ی موسیقی به کاربران طراحی کرد.
</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
داده‌های آموزشی در فایل <code dir=ltr>amazon_train.csv</code> در دسترس شما قرار گرفته است. این فایل شامل ۴ ستون است که به‌ترتیب بیانگر شناسه‌ی کاربر، شناسه‌ی محصول، امتیازی که کاربر به محصول داده و زمان ثبت امتیاز هستند. شبکه‌ای که در ادامه پیاده خواهیم کرد شناسه‌ی عددی کاربر و محصول را ورودی می‌گرد و امتیازی که کاربر به آن محصول می‌دهد را تخمین خواهد زد. ابتدا فایل مجموعه‌داده را به‌کمک <code>pandas</code> بخوانید.
</font>
</p>

In [2]:
import numpy as np
import pandas as pd

# Loading the dataset
train = pd.read_csv('/content/amazon_train.csv')
val = pd.read_csv('/content/amazon_valid.csv')
test = pd.read_csv('/content/amazon_test.csv')

In [3]:
train.shape

(909201, 4)

In [4]:
val.head()

,UserID,ProductID,Rating,Timestamp,UserEnc
0,A3SBTW3WS4IQSN,B005FDXMJS,1.0,2014-07-23,66384
1,A1NXNMRAVPGTO3,B00F9FCW7K,1.0,2014-07-23,15871
2,A1VP1KVX7Y4PHQ,B00FEE7B1I,4.0,2014-07-22,20933
3,A1XEB94731LGAR,B007VGGIRK,5.0,2014-07-22,22067
4,A2I3XS9TO93Q0F,B00JCE14AU,4.0,2014-07-22,35766


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
پس از بررسی و تحلیل داده‌هایی که در اختیارتان گرفته ممکن است به پیش‌پردازش‌های مختلفی نیاز پیدا کنید و ایده‌های گوناگونی را پیاده کنید. با این حال، دو پیش‌پردازش اصلی که حتماً به آن‌ها نیاز خواهید داشت را در ادامه شرح داده‌ایم.
</font>
</p>

<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
کدگذاری
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
از آن‌جا که شناسه‌های کاربر (<code dir=ltr>UserID</code>) و محصول (<code dir=ltr>ProductID</code>) به‌صورت رشته‌ای هستند، نیاز است آن‌ها را به‌شکل اعداد مرتب کدگذاری کنیم. بنابراین ابتدا با استفاده از کلاس <code>LabelEncoder</code> موجود در کتابخانه‌ی <code>sklearn</code> هر کدام از این ستون‌ها را کدگذاری کنید. پیشنهاد می‌شود حاصل را در ستون‌های مجزایی به‌نام‌های <code>UserEnc</code> و <code>ProductEnc</code> ذخیره کنید.
</font>
</p>

In [5]:
from sklearn.preprocessing import LabelEncoder

# Encoding each ID string
le = LabelEncoder()
train['ProductEnc'] = le.fit_transform(train['ProductID'])
val['ProductEnc'] = le.transform(val['ProductID'])
test['ProductEnc'] = le.transform(test['ProductID'])

In [6]:
le1 = LabelEncoder()
train['UserEnc'] = le1.fit_transform(train['UserID'])

<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
نرمال‌سازی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
اکنون مقادیر ستون <code>Rating</code> را به‌کمک <code>MinMaxScaler</code> کتابخانه‌ی <code>sklearn</code> نرمال کنید تا مقادیر آن به بازه‌ی ۰ تا ۱ نگاشت شود.
</font>
</p>

In [7]:
# Scaling the Rating column
train['Rating'] = train['Rating'] / 5
val['Rating'] = val['Rating'] / 5
train.head(10)

,UserID,ProductID,Rating,Timestamp,ProductEnc,UserEnc
0,A3HICVLF4PFFMN,0594481813,1.0,2014-05-05,0,59277
1,A2QBZA4S1ROX9Q,0594481813,0.6,2013-05-25,0,41167
2,AT09WGFUM934H,0594481813,0.6,2013-08-31,0,85549
3,AGAKHE014LQFU,0594481813,0.6,2013-09-18,0,77047
4,A1S6B5QFWGVL5U,0594481813,0.8,2013-06-27,0,18672
5,A2IDCSC6NVONIZ,0972683275,1.0,2013-04-30,1,35949
6,A3BMUBUC1N77U8,0972683275,0.8,2013-11-23,1,55248
7,AVRFGGCCCR6QU,0972683275,0.8,2010-08-30,1,87318
8,AYQNWE3AX4H08,0972683275,1.0,2013-07-18,1,89229
9,A3P24ZSF9IQRJJ,0972683275,1.0,2014-06-13,1,64167


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
حال نوبت به مدل‌سازی می‌رسد. ساختار پیشنهادی ما به این شکل است که شبکه دو ورودی خواهد داشت: شناسه‌ی کاربر و شناسه‌ی کالا. برای هر کدام از این شناسه‌ها یک لایه‌ی تعبیه‌ساز (<code>Embedding</code>) جداگانه در نظر می‌گیریم تا به‌ازای هر کاربر یا کالا، یک بردار تعبیه‌ی بامعنا ساخته شود. خروجی هر لایه‌ی تعبیه به‌کمک یک لایه‌ی <code>Flatten</code> مسطح خواهد شد. پس از این مراحل، دو بردار بازنمایی کالا و کاربر در اختیار شما قرار می‌گیرد و می‌توانید پس از آن معماری دلخواه خود را پیاده کنید. به‌عنوان مثال می‌توانید بردار تعبیه‌ی کالا را در بردار تعبیه‌ی کاربر ضرب نقطه‌ای کنید (به‌کمک لایه‌ی <code>dot</code>) و حاصل را از تابع فعال‌ساز <code>sigmoid</code> عبور دهید تا خروجی شبکه یعنی امتیاز کاربر به آن کالا بین ۰ تا ۱ تولید شود. بنابراین ابتدا این ساختار را به‌کمک مدل تابعی (Functional) کراس پیاده‌سازی کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="orange"><b>نکته:</b></font>
<font face="vazir" size=3>
دست شما کاملاً برای ایجاد تغییرات در این ساختار باز است. به‌عنوان مثال می‌توانید علاوه بر تولید بردارهای تعبیه، به‌کمک لایه‌های اضافه‌ی <code>Embedding</code> (با اندازه‌ی بردار ۱) یک مقدار بایاس هم برای هر کاربر یا کالا تولید کنید. در نهایت می‌توانید بایاس کاربر، بایاس کالا و حاصل ضرب نقطه‌ای را به‌کمک یک لایه‌ی <code>add</code> جمع کنید. به‌عنوان ایده‌ای دیگر می‌توانید به‌جای آن‌که دو بردار تعبیه را در یکدیگر ضرب کنید از عملیات‌های دیگری همچون الحاق (<code>Concatenate</code>) استفاده کنید. همچنین می‌توانید بر تعداد لایه‌ها بی‌افزایید، یا روش مقداردهی اولیه‌ی وزن‌ها یا رگولاریزیشن هر لایه را تنظیم کنید.
</font>
</p>

In [8]:
train['ProductEnc'].max()

48768

In [9]:
import keras

In [10]:
# Building the model
input = keras.Input(shape= (2,))
x1 = keras.layers.Embedding(input_dim= 48769, output_dim= 64)(input[:, 0])
x2 = keras.layers.Embedding(input_dim= 90050, output_dim= 64)(input[:, 1])
x = x1 * x2
output = keras.layers.Dense(1, activation= 'sigmoid')(x)
model = keras.Model(inputs=input, outputs=output)

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
پس از طراحی معماری شبکه، می‌توانید آن را با ورودی دادن یک لیست شامل دو لیست شناسه‌ی کاربر و شناسه‌ی کالا آموزش دهید. متغیر هدف نیز همان ستون <code>Rating</code> خواهد بود. همچنین در صورتی‌که در لایه‌ی آخر از تابع فعال‌ساز <code>sigmoid</code> برای نورون خروجی استفاده کرده باشید می‌توانید از تابع زیان <code>binary_crossentropy</code> بهره ببرید. در صورت علاقه و نیاز می‌توانید از هر تکنیک دلخواهی در فرآیند آموزش مدل خود استفاده کنید.
</font>
</p>

In [11]:
# Training phase
model.compile(optimizer= 'adam',
              loss= 'binary_crossentropy',
              metrics= ['accuracy'])

In [12]:
train.head()

,UserID,ProductID,Rating,Timestamp,ProductEnc,UserEnc
0,A3HICVLF4PFFMN,0594481813,1.0,2014-05-05,0,59277
1,A2QBZA4S1ROX9Q,0594481813,0.6,2013-05-25,0,41167
2,AT09WGFUM934H,0594481813,0.6,2013-08-31,0,85549
3,AGAKHE014LQFU,0594481813,0.6,2013-09-18,0,77047
4,A1S6B5QFWGVL5U,0594481813,0.8,2013-06-27,0,18672


In [14]:
model.fit(np.array(train[['ProductEnc', 'UserEnc']]), train['Rating'],
          batch_size= 32,
          epochs= 5)

Epoch 1/5
28413/28413 ━━━━━━━━━━━━━━━━━━━━ 112s 4ms/step - accuracy: 0.5987 - loss: 0.4430
Epoch 2/5
28413/28413 ━━━━━━━━━━━━━━━━━━━━ 142s 4ms/step - accuracy: 0.5984 - loss: 0.3383
Epoch 3/5
28413/28413 ━━━━━━━━━━━━━━━━━━━━ 140s 4ms/step - accuracy: 0.5991 - loss: 0.2653
Epoch 4/5
28413/28413 ━━━━━━━━━━━━━━━━━━━━ 141s 4ms/step - accuracy: 0.5986 - loss: 0.2536
Epoch 5/5
28413/28413 ━━━━━━━━━━━━━━━━━━━━ 109s 4ms/step - accuracy: 0.5982 - loss: 0.2453


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌بینی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
دو فایل دیگر به نام‌های <code>amazon_valid.csv</code> و <code>amazon_test.csv</code> نیز به‌عنوان مجموعه‌ی اعتبارسنجی و آزمون در اختیار شما قرار گرفته است. مدل شما باید بتواند به‌ازای هر جفت کاربر و کالا، امتیازی که آن کاربر به آن کالا می‌دهد را به‌درستی تخمین بزند. ساختار این مجموعه‌ها نیز مشابه با مجموعه‌ی آموزشی است با این تفاوت که برچسب حقیقی داده‌های آزمون در دسترس شما نیست و ارزیابی مدل شما بر روی داده‌های آزمون در سامانه‌ی داوری کوئرا انجام می‌پذیرد. با این حال، توزیع داده‌ها بین مجموعه‌ی اعتبارسنجی و آزمون بسیار مشابه است و اگر مدل شما برای نمونه‌های اعتبارسنجی به عملکرد قابل قبولی برسد به احتمال زیاد برای نمونه‌های آزمون نیز همین‌گونه خواهد بود.
</font>
</p>


<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
هدفی که در این نوع مسئله در نظر گرفته‌ایم این است که مدل شما تا چه‌اندازه می‌تواند کالاهایی که کاربر به آن‌ها امتیاز بالا (۴ یا ۵) می‌دهد را به‌درستی تشخیص دهد، بنابراین
عملکرد مدل شما در سامانه‌ی داوری به این‌گونه ارزیابی خواهد شد که برچسب‌های <code>5</code> و <code>4</code> به‌عنوان دسته‌ی مثبت و سایر برچسب‌ها به‌عنوان دسته‌ی منفی در نظر گرفته می‌شود.
سپس از معیار <code>Precision</code> جهت ارزیابی مدل شما استفاده خواهد شد که در واقع نسبت تعداد پیشنهادات مثبت درست (مدل تشخیص داده که کاربر به کالا امتیاز بالا می‌دهد و واقعاً این‌گونه بوده) به کل پیشنهادات مثبت مدل را می‌شمارد.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazir" size=3>
 جهت کسب امتیاز کامل نیاز است تا پاسخ شما حداقل امتیاز <code>80</code> را با توجه به معیار معرفی‌شده کسب نماید.
</font>
</p>

In [ ]:
# Evaluate on validation data
# TODO

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
ساختار خروجی برای نمونه‌های آزمون
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
 پیش‌بینی مدل خود را برای نمونه‌های آزمون به‌شکل امتیازی بین ۱ تا ۵ تولید کرده و در ستون <code>Rating</code> یک دیتافریم با نام <code>submission</code> در قالب جدول زیر ذخیره کنید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>UserID</code>|شناسه‌ی کاربر (مشابه با نمونه‌ی آزمون)|
|<code>ProductID</code>|شناسه‌ی کالا (مشابه با نمونه‌ی آزمون)|
|<code>Rating</code>|امتیاز کاربر به کالا از ۱ تا ۵ (پیش‌بینی مدل)|
    
</font>
</div>
</center>

In [18]:
test.shape

(89732, 5)

In [22]:
submission = pd.DataFrame()
a = (model.predict(np.array(test[['ProductEnc', 'UserEnc']])))

2805/2805 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


In [27]:
a.reshape(89732)

array([0.8827383 , 0.742449  , 0.78472304, ..., 0.82299775, 0.9159428 ,
       0.81520414], dtype=float32)

In [29]:
# Preparing submission file
submission['Rating'] = (a.reshape(89732)) * 5 # TODO
submission['UserID'] = test['UserID']
submission['ProductID'] = test['ProductID']
submission

,Rating,UserID,ProductID
0,4.413692,A2Y3A341VDK37H,B00HFI55N2
1,3.712245,A240FRPD4MEXND,B00KIMX4EY
2,3.923615,A3SBTW3WS4IQSN,B001IF252M
3,4.555170,A1QCZQTJNK3MU,B00J46VVKE
4,3.187538,A16JT7E5121OAB,B00JZC972Q
...,...,...,...
89727,4.188495,A37Z65SZVT0TVB,B00005AR4L
89728,4.855173,A3PLX6PTM2ERKL,B00004Z6PI
89729,4.114989,A3TRPVAGT3NWBS,B00005NVPW
89730,4.579714,AY8Q1X7G96HV5,B000059MSI


In [34]:
submission['Rating'] = submission['Rating'].astype(int)
submission

,Rating,UserID,ProductID
0,4,A2Y3A341VDK37H,B00HFI55N2
1,3,A240FRPD4MEXND,B00KIMX4EY
2,3,A3SBTW3WS4IQSN,B001IF252M
3,4,A1QCZQTJNK3MU,B00J46VVKE
4,3,A16JT7E5121OAB,B00JZC972Q
...,...,...,...
89727,4,A37Z65SZVT0TVB,B00005AR4L
89728,4,A3PLX6PTM2ERKL,B00004Z6PI
89729,4,A3TRPVAGT3NWBS,B00005NVPW
89730,4,AY8Q1X7G96HV5,B000059MSI


In [39]:
submission['Rating'] = submission['Rating'] + 1

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد. همچنین اگر از گوگل کولب استفاده می‌کنید، در صورت نیاز به پشتیبانی حتماً آخرین نسخه از نت‌بوک را به‌صورت دستی دانلود کرده و داخل فایل ارسالی قرار دهید یا لینک کولب را با ما به‌اشتراک بگذارید.
</font>
</p>

In [40]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'what_to_buy.ipynb')):
    %notebook -e what_to_buy.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['what_to_buy.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['what_to_buy.ipynb', 'submission.csv']


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
بیشتر بدانید:
سیستم پیشنهاددهنده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
برای پیشنهاد کالای جدید به یک کاربر روش‌های مختلفی وجود دارد، اما یک راه ساده این است که از تخمین امتیازی که مدل به‌ازای هر جفت کاربر و کالا می‌دهد استفاده کنیم.
به‌ازای یک کاربر به‌کمک مدل خود تخمین می‌زنیم که وی از بین کالاهایی که تاکنون خریداری نکرده، کدام‌ها را بیشتر از همه دوست خواهد داشت و امتیاز بالاتری به آن‌ها خواهد داد. بنابراین می‌توانیم این کالاها را به او پیشنهاد دهیم و به‌احتمال زیاد از تعدادی از آن‌ها خوشش خواهد آمد. البته این موضوع نمی‌تواند مستقیماً به این معنی باشد که آن کالا را خریداری می‌کند. به‌عنوان مثال شاید یک کالای گران‌قیمت را پیشنهاد دهیم که همگی از آن خوشمان می‌آید اما بسیاری قدرت خرید آن را نداریم! 🫠
</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
<span style="color:green"><b>راهنمایي:</b></span>
برای تولید پیش‌بینی به‌ازای یک کاربر ابتدا لیستی از کالاهایی که وی تاکنون خریداری نکرده را پیدا کنید. سپس جفت شناسه‌ی کدگذاری‌شده‌ی کاربر و هر یک از این کالاها را به مدل ورودی بدهید تا امتیاز کاربر به هر کدام از آن کالاها تخمین زده شود. در نهایت می‌توان <code>n</code> کالایی که منجر به بیشترین امتیاز (خروجی مدل) شده را انتخاب کرد و به وی پیشنهاد داد.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
<span style="color:purple"><b>🧐 بیشتر بی‌اندیشیم:</b></span>
چه روش‌های دیگری جهت پیشنهاد کالا به یک کاربر مشخص به ذهن‌تان می‌رسد؟ آیا می‌توان مستقیماً‌ از بردارهای تعبیه‌ی هر کالا یا کاربر جهت پیشنهاد استفاده کرد؟ به چه روش‌های می‌توان کاربران مشابه با یک کاربر را پیدا کرده و براساس تجربه یا الگوی خرید آن‌ها به کاربر کالاهایی را پیشنهاد داد؟ پیشنهاد می‌کنیم به این موضوعات بیشتر فکر کرده و نظرات خود را از طریق بخش دیدگاه عمومی صفحه‌ی پروژه در کوئرا با دیگران به‌اشتراک بگذارید. چه بهتر که ایده‌های خود را به‌شکل عملی نیز آزمایش کنید.
</font>
</p>